In [1]:
computer = 'win'

if computer == 'mac':
    str_chromedriver = '../../../chromedriver'
elif computer == 'win':
    str_chromedriver = '../../../chromedriver.exe'
else:
    print('？？？')
    exit(0)

In [2]:
import pandas as pd

In [3]:
df_menu = pd.read_excel('./kit.xlsx',
                        header=0,
                        dtype=str).fillna('')

list_url = df_menu['Kit_Info'].tolist()

print('总数量：' + str(len(list_url)))
list_url

总数量：2439


['https://www.rockauto.com/en/moreinfo.php?pk=8921164&cc=0&pt=5724',
 'https://www.rockauto.com/en/moreinfo.php?pk=8921168&cc=0&pt=5724',
 'https://www.rockauto.com/en/moreinfo.php?pk=8921288&cc=0&pt=5736',
 'https://www.rockauto.com/en/moreinfo.php?pk=9659104&cc=0&pt=5736',
 'https://www.rockauto.com/en/moreinfo.php?pk=9659108&cc=0&pt=5736',
 'https://www.rockauto.com/en/moreinfo.php?pk=9722104&cc=0&pt=5728',
 'https://www.rockauto.com/en/moreinfo.php?pk=9722108&cc=0&pt=5728',
 'https://www.rockauto.com/en/moreinfo.php?pk=9854520&cc=0&pt=5728',
 'https://www.rockauto.com/en/moreinfo.php?pk=8306460&cc=0&pt=5724',
 'https://www.rockauto.com/en/moreinfo.php?pk=8306464&cc=0&pt=5724',
 'https://www.rockauto.com/en/moreinfo.php?pk=8307124&cc=0&pt=5728',
 'https://www.rockauto.com/en/moreinfo.php?pk=8307492&cc=0&pt=5728',
 'https://www.rockauto.com/en/moreinfo.php?pk=8307496&cc=0&pt=5728',
 'https://www.rockauto.com/en/moreinfo.php?pk=8308212&cc=0&pt=5736',
 'https://www.rockauto.com/en/more

In [4]:
df = pd.DataFrame()

df

""


In [5]:
from selenium.webdriver import ChromeOptions

In [6]:
option = ChromeOptions()
option.add_experimental_option('useAutomationExtension', False)
# option.add_argument('blink-settings=imagesEnabled=false')
option.add_argument('--disable-blink-features=AutomationControlled')
option.add_argument('--disable-dev-shm-usage')
option.add_argument('--disable-gpu')
option.add_experimental_option('excludeSwitches', ['enable-automation'])
# option.add_argument('--headless')
option.add_argument('--no-sandbox')
# option.add_argument('--proxy-server=' + Proxy.get_Proxy_selenium())
# option.add_argument('user-agent=' + UA.get_UA())
option.page_load_strategy = 'eager'

In [7]:
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.service import Service

In [8]:
browser = Chrome(service=Service(str_chromedriver), options=option)

browser.maximize_window()

In [9]:
import time

In [10]:
url = list_url[0]

browser.get(url)

time.sleep(3)

In [11]:
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By

In [12]:
wait = WebDriverWait(browser, 60)
wait.until(EC.presence_of_element_located((By.XPATH, '//section[contains(@aria-label, "More Information for")]')))
list_oe = browser.find_elements(by=By.XPATH, value='//section[contains(@aria-label, "Alternate/OEM Part Number")]')

oe = ''
if len(list_oe) != 0:
    for _ in list_oe:
        if _.text.startswith('{Alternate Inventory Numbers: '):
            oe += _.text.replace('{Alternate Inventory Numbers: ', '').replace('}', '') + ';'
        elif _.text.startswith('OEM / Interchange Numbers: '):
            oe += _.text.replace('OEM / Interchange Numbers: ', '') + ';'
            
oe = ';'.join(oe[:-1].split(', '))

oe

'135060P010;135060P011;1350631020;1350631030;1350631031'

In [13]:
src = browser.find_elements(by=By.XPATH, value='//img[@id="inlineimg[1]"]')
if len(src) == 0:
    src = ''
else:
    src = src[0].get_attribute('src')

src

'https://www.rockauto.com/info/583/1025-1-ANG.jpg'

In [14]:
df_temp = pd.DataFrame([{'status': 'ok',
                         'Kit_Info': url,
                         'Kit_OE': oe,
                         'Kit_Src': src}])

browser.quit()

df = pd.concat([df, df_temp], ignore_index=True).fillna('')

df

,status,Kit_Info,Kit_OE,Kit_Src
0,ok,https://www.rockauto.com/en/moreinfo.php?pk=89...,135060P010;135060P011;1350631020;1350631030;13...,https://www.rockauto.com/info/583/1025-1-ANG.jpg
